In [ ]:
input_images = X[y==2][0:100]
H = theano.shared(
    #np.random.normal(size=(input_images.shape[0], latent_size)).astype(np.float32)
    capsule.get_latent(input_images)
)

In [ ]:
input_variables = OrderedDict()
input_variables["input_image"] = dict(tensor_type=T.matrix)
class ModelReverseGeneration:
    def get_all_params(self, **t):
        return [H]

model_reverse =   ModelReverseGeneration()
model_reverse.x_to_z = x_to_z
model_reverse.z_to_x = z_to_x

def loss_function_reverse(model, tensors):
    input_image = tensors["input_image"]    
    output = get_output_from_latent(model, H)    
    recons = ((input_image - output) ** 2).sum(axis=1).mean()
    print(recons.ndim)
    return recons

batch_optimizer = MyBatchOptimizer(
    verbose=1,
    max_nb_epochs=2000,
    batch_size=100,
    optimization_procedure=(updates.adadelta, 
                            {"learning_rate": 0.01})
)

capsule_reverse = Capsule(
    input_variables, 
    model_reverse,
    loss_function_reverse,
    functions=functions,
    batch_optimizer=batch_optimizer,
)

In [ ]:
capsule_reverse.fit(input_image=input_images)

In [ ]:
output_images = theano.function([], get_output_from_latent(model_reverse, H))()
fig = plt.figure(figsize=(10, 10))
n = 4
gridw, gridh = n*2, n*2
k = 1
ind = 0
for i in range(n):
    for j in range(n):
        if ind >= input_images.shape[0]:
            break
        plt.subplot(gridw, gridh, k)
        plt.axis('off')
        plt.imshow(input_images[ind].reshape((w, h)), cmap="gray")
        k += 1
        plt.subplot(gridw, gridh, k)
        plt.axis('off')
        plt.imshow(output_images[ind].reshape((w, h)), cmap="gray")
        k += 1
        ind += 1
plt.show()